In [ ]:
!pip install -U ipywidgets
!pip install ultralytics
!pip install os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os
from matplotlib import pyplot as plt
from PIL import Image
import torch
from torch import nn
import yaml
import shutil

In [ ]:
from ultralytics import YOLO
print("All libraries have been imported", end='\r')

In [ ]:
model = YOLO('yolov8s.pt')

100%|██████████| 21.5M/21.5M [00:00<00:00, 143MB/s] 


In [ ]:
train_data ="/content/drive/MyDrive/kaggle/input/car-object-detection/data/training_images"
csv_data = "/content/drive/MyDrive/kaggle/input/car-object-detection/data/train_solution_bounding_boxes (1).csv"
test_data ="/content/drive/MyDrive/kaggle/input/car-object-detection/data/testing_images"

In [ ]:
!mkdir -p "/kaggle/working/data"

!mkdir -p "/kaggle/working/data/images"

!mkdir -p "/kaggle/working/data/images/train"
!mkdir -p "/kaggle/working/data/images/val"

!mkdir -p "/kaggle/working/data/labels"

!mkdir -p "/kaggle/working/data/labels/train"
!mkdir -p "/kaggle/working/data/labels/val"

In [ ]:
df=pd.read_csv(csv_data)

width=676
height=380

df["class"]=0
df.rename(columns={'image':'img_name'}, inplace=True)

df["x_centre"]=(df["xmin"]+df["xmax"])/2
df["y_centre"]=(df["ymin"]+df["ymax"])/2
df["width"]=(df["xmax"]-df["xmin"])
df["height"]=(df["ymax"]-df["ymin"])

# normalize the bounding box coordinates
df["x_centre"]=df["x_centre"]/width
df["y_centre"]=df["y_centre"]/height
df["width"]=df["width"]/width
df["height"]=df["height"]/height

df_yolo=df[["img_name","class","x_centre","y_centre","width","height"]]
df_yolo.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/kaggle/input/car-object-detection/data/train_solution_bounding_boxes (1).csv'

In [ ]:
root_dir="/kaggle/working/data"
labels_dir="/kaggle/working/data/labels"
images_dir="/kaggle/working/data/images"

In [ ]:
img_list = list(sorted(os.listdir(train_data)))
np.random.shuffle(img_list)

In [ ]:
for i, img_name in enumerate(img_list):
    subset = "train"
    if i >= 80/100 * len(img_list):
        subset = "val"


    if np.isin(img_name, df_yolo['img_name']):
        columns = ["class", "x_centre", "y_centre", "width", "height"]
        img_box = df[df['img_name'] == img_name][columns].values
        label_path = os.path.join(labels_dir,subset, img_name[:-4] + ".txt")
        with open(label_path, "w+") as f:
            for row in img_box:
                text = " ".join(row.astype(str))
                f.write(text)
                f.write("\n")

    old_image_path = os.path.join(train_data, img_name)
    new_image_path = os.path.join(images_dir, subset, img_name)
    shutil.copy(old_image_path, new_image_path)

In [ ]:
yolo_format=dict(path="/kaggle/working/data",
                 train="/kaggle/working/data/images/train",
                 val="/kaggle/working/data/images/val",
                 nc=1,
                 names={0:"car"})

with open('/kaggle/working/data.yaml', 'w') as outfile:
    yaml.dump(yolo_format, outfile, default_flow_style=False)

In [ ]:
#Train The Model

In [ ]:
results = model.train(data='/kaggle/working/data.yaml', epochs=5, imgsz=640)

In [ ]:
metrics = model.val()

In [ ]:
#Plotting The Curves

In [ ]:
list_of_metrics = ["P_curve.png","R_curve.png","confusion_matrix.png", "F1_curve.png", "results.png"]

In [ ]:
for i in list_of_metrics:

    image = Image.open(f'/kaggle/working/runs/detect/train/{i}')

    plt.figure(figsize=(12, 8))
    plt.axis("off")
    plt.imshow(image)
    plt.show()

In [ ]:
df=pd.read_csv(csv_data)
print(df.head())

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

fig, ax = plt.subplots()

for index, row in df.iterrows():
    # Assuming your dataset's columns are named 'xmin', 'ymin', 'xmax', 'ymax'
    # Replace these with the actual column names
    xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']

    # Calculate width and height
    width, height = xmax - xmin, ymax - ymin

    # Create a rectangle patch
    rect = patches.Rectangle((xmin, ymin), width, height, linewidth=1, edgecolor='r', facecolor='none')

    # Add the rectangle to the plot
    ax.add_patch(rect)

# Set limits if necessary (optional, depending on your dataset's range)
ax.set_xlim([df['xmin'].min() - 10, df['xmax'].max() + 10])
ax.set_ylim([df['ymin'].min() - 10, df['ymax'].max() + 10])

plt.show()